In [21]:
import os
import xml.etree.ElementTree as ET


# 폴더 경로 설정
folder_path = r"C:\VISSIM_Workspace\test3"  # 폴더 경로 입력 받아야함 추후에 함수에 파라미터로 처리 예정
# 폴더 내 파일 목록 가져오기
file_list = os.listdir(folder_path)
inpx_files = [file for file in file_list if file.endswith(".inpx")]

for i in range(1):
    file_name = inpx_files[i]
    print("file_name : ", file_name)
    file_path = folder_path + "\\" +file_name
    print("file_path : ", file_path)
    tree =  ET.parse(file_path)
    root = tree.getroot()

    # <vehicleCompositions> 태그 내 <vehicleComposition name="Default"> 찾기
    for vehicle_comp in root.findall(".//vehicleComposition"):
        if vehicle_comp.get("name") == "test":
            print(f"Found <vehicleComposition> with name: {vehicle_comp.get('name')}")

            # <vehCompRelFlows> 태그 찾기
            vehCompRelFlows = vehicle_comp.find("vehCompRelFlows")
            if vehCompRelFlows is not None:
                # <vehicleCompositionRelativeFlow> 태그 찾기
                for relflow in vehCompRelFlows.findall("vehicleCompositionRelativeFlow"):
                    relflow_value = relflow.get("relFlow")
                    vehType = relflow.get("vehType")
                    if(vehType == "200"):
                        print(f"vehType: {vehType}, relFlow: {relflow_value}")

    # 변경된 XML을 원본 파일에 덮어쓰기
    #tree.write(file_path, encoding="utf-8", xml_declaration=True)
    #print(f"파일이 수정되어 저장되었습니다: {file_path}")

file_name :  test.inpx
file_path :  C:\VISSIM_Workspace\test3\test.inpx
Found <vehicleComposition> with name: test
vehType: 200, relFlow: 0.05


In [ ]:
"